In [2]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
from sklearn.model_selection import StratifiedKFold
import os
from sklearn.preprocessing import StandardScaler
from random import sample
import random
import matplotlib.pyplot as plt

# load data

In [109]:
feature = pd.read_csv('./result/featuresV1.csv')
feature = feature[['alert_key', 'trade_amount_usd', 'amt_sum',
       'amt_std', 'd_cur_type', 'day', 'trans_no_final', 'd_cur_type_ratio',
       'sum_0', 'sum_1', 'var_0', 'var_1', 'transnation_rate',
       'total_amount_money_in', 'total_amount_money_out', 'var_money_in',
       'var_money_out', 'cross_bank_rate', 'risk_rank',
       'occupation_code', 'total_asset', 'AGE', 'lupay_max', 'lupay_std',
       'clamt_max', 'clamt_std', 'csamt_max', 'csamt_std', 'inamt_max',
       'inamt_std', 'cucsm_max', 'cucsm_std', 'cucah_max', 'cucah_std',
       'cytocam_times', 'cytocam_first', 'cytocam_last', 'cytocam_variance']]

In [110]:

target = pd.read_csv('./data/train_y_answer.csv')
feature_public = pd.DataFrame([])
for i in range(len(feature)):
    if feature['alert_key'].iloc[i] not in list(target.alert_key):
        dp_t = pd.DataFrame(np.array([np.array(feature.iloc[i])]), columns = feature.columns)
        feature_public = pd.concat([feature_public, dp_t])
target.set_index('alert_key', inplace = True)
feature.set_index('alert_key', inplace = True)

feature = feature.reindex(target.index)
feature = pd.concat([feature, target], axis = 1)
alert_key = feature_public.alert_key
feature_public.set_index('alert_key', inplace = True)
feature_public = feature_public.fillna(0)
feature = feature.fillna(0)

In [111]:
df = feature
df_public = feature_public

# Random Forest model

In [112]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators = 100)
rf_clf.fit(df.iloc[:, :-1], df.iloc[:, -1])
# y_this_ = rf_clf.predict_proba(df_public)

RandomForestClassifier()

# XGBoost model

In [113]:
from xgboost import XGBClassifier
random.seed(100)

xgboostModel = XGBClassifier(n_estimators=100, learning_rate= 0.25)
xgboostModel.fit(df.iloc[:, :-1], df.iloc[:, -1])
# y_this_ = xgboostModel.predict_proba(df_public)


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.25, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0, ...)

# SVM model

In [114]:
from sklearn.svm import SVC
svm_clf = SVC(probability = True, random_state = 100)
svm_clf.fit(df.iloc[:, :-1], df.iloc[:, -1])
# y_this_ = svm_clf.predict_proba(df_public)

SVC(probability=True, random_state=100)

# voting classifier

In [115]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(estimators = [ ('rf_clf', rf_clf), ('svm_clf', svm_clf), ('xgboostModel', xgboostModel)], voting='soft', weights = [0,7,3])

In [116]:
clf = voting_clf
clf.fit(df.iloc[:, :-1], df.iloc[:, -1])
y_this_ = clf.predict_proba(df_public)

In [117]:
def score(y_true: pd.DataFrame, y_pred: pd.DataFrame):
    """ 
    y_true - pandas.DataFrame: alert_key | sar_flag
    y_pred - pandas.DataFrame: alert_key | prob  
    """
    n = len(y_true[y_true.sar_flag==1]) # Num sar
    sar = y_true[y_true.sar_flag==1].alert_key.values # sar alert keys
    y_pred.sort_values(by=['probability'], ascending=False, inplace=True)
    hit = 0
    for i, y_pred_ in enumerate(y_pred.alert_key.values):
        if y_pred_ in sar:
            hit += 1
        if hit == n-1:
            break
    return (n-1)/(i+1), n-1, i

# public score

In [118]:
answer = pd.read_csv('./data/24_ESun_public_y_answer.csv')
answer.set_index('alert_key', inplace = True)
y_this_ = pd.DataFrame(y_this_)
a = pd.concat([pd.DataFrame(np.array([np.array(alert_key)]).T), y_this_[y_this_.columns[1]]], axis = 1)
a.columns = ['alert_key', 'probability']
a.set_index('alert_key', inplace = True)
a = a.reindex(answer.index)
a.reset_index(inplace= True)
answer.reset_index(inplace = True)
score_ = score(answer, a)
print(score_[0])

0.01631321370309951


# SAR prob rank

In [19]:
y_this_ = pd.DataFrame(y_this_)
a = pd.concat([pd.DataFrame(np.array([np.array(alert_key)]).T), y_this_[y_this_.columns[1]]], axis = 1)
a.columns = ['alert_key', 'probability']
a = a.sort_values('probability', ascending = False)
a.reset_index(inplace = True)
a.drop(columns = 'index', inplace = True)
answer = pd.read_csv('./data/24_ESun_public_y_answer.csv')
answer_1 = answer[answer.sar_flag == 1]
for i in answer_1['alert_key']:
    index = a[a.alert_key == i].index
    print(i, index.values)

354939 [71]
355091 [97]
355152 [1398]
355724 [65]
359668 [431]
356602 [33]
363320 [30]
358453 [612]
363896 [324]
361617 [574]
363033 [398]


In [20]:
y_this_ = pd.DataFrame(y_this_)
p = pd.concat([pd.DataFrame(np.array([np.array(alert_key)]).T), y_this_[y_this_.columns[1]]], axis = 1)
p.columns = ['alert_key', 'probability']
sample = pd.read_csv('./data/sample.csv')
sample.set_index('alert_key', inplace = True)
p.set_index('alert_key', inplace = True)
p = p.reindex(sample.index)
p.reset_index(inplace = True)
p = p.fillna(0)
p.columns = ['alert_key', 'probability']
p.to_csv('./result/Public_Prob_FeatureV1.csv', index = False)

/homes/jingchun/.conda/envs/torch1.11_jingchun/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
